In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install catboost
!pip install lightgbm
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 538.2/538.2 kB 8.9 MB/s eta 0:00:00


In [ ]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Essentials
import numpy as np
import pandas as pd
import random
import time
import gc
import os
from datetime import datetime

# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns
# matplotlib 의 기본 scheme 말고
# seaborn scheme 을 세팅하여, 일일이 graph의 font size 지정할 필요 없이
# seaborn 의 font_scale 을 사용하는 것을 추천드립니다.
plt.style.use('seaborn')
sns.set(font_scale=2.5)
from matplotlib.pylab import rcParams
##set up the parameters
rcParams['figure.figsize'] = 80,60

# Models
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
import lightgbm
import xgboost as xgb
import catboost
from xgboost import plot_tree


# Misc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV # 최상의 하이퍼 파라미터를 선정해주는 라이브러리
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, cross_val_score


pd.set_option('display.max_columns', None)

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000

<Figure size 1600x800 with 0 Axes>

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/model/4.24_land_modi(개별).csv'
df = pd.read_csv(path)
df

,Unnamed: 0,annual_inc,dti,installment,int_rate,loan_amnt,mort_acc,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,num_accts_ever_120_pd,pub_rec,pub_rec_bankruptcies,loan_status,emp_length_int,issue_d_year,issue_d_month,grade_numeric,fico_range_mean,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,term_ 60 months,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,1,94000.0,17.41,822.95,14.47,35000.0,0.0,30.0,154.0,0.0,0.0,0.0,Current,2.0,2018,9,5,717.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,110000.0,34.70,680.82,24.99,23200.0,2.0,126.0,277.0,0.0,0.0,0.0,Charged Off,10.0,2016,7,3,672.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,3,35000.0,14.95,145.06,9.93,4500.0,0.0,127.0,265.0,1.0,0.0,0.0,Current,10.0,2017,12,6,677.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5,65000.0,17.74,494.55,7.07,16000.0,3.0,134.0,400.0,0.0,0.0,0.0,Charged Off,0.0,2017,10,7,822.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,6,50000.0,6.99,146.10,10.42,4500.0,1.0,226.0,130.0,0.0,2.0,2.0,Fully Paid,5.0,2017,5,6,702.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535702,1755290,33800.0,19.18,133.78,19.99,3600.0,1.0,151.0,36.0,0.0,0.0,0.0,Charged Off,7.0,2014,3,3,667.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1535703,1755291,71000.0,23.92,583.89,6.68,19000.0,5.0,125.0,216.0,0.0,0.0,0.0,Fully Paid,4.0,2015,4,7,737.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1535704,1755292,44000.0,12.85,329.65,14.33,9600.0,1.0,25.0,99.0,0.0,0.0,0.0,Fully Paid,9.0,2013,5,5,692.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1535705,1755293,67000.0,20.94,744.90,7.35,24000.0,2.0,54.0,151.0,1.0,0.0,0.0,Fully Paid,5.0,2018,2,7,737.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:

# target 만들기
not_deliq = ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']

loan_stat = df['loan_status']
target = []
for stat in loan_stat:
    # 연체 하지 않은 경우를 0
    if stat in not_deliq:
        target.append(0)
    # 연체한 경우를 1
    else:
        target.append(1)
print(len(target))
print(len(df))

1535707
1535707


In [ ]:
df['target'] = target

In [ ]:
df.drop(columns = 'loan_status', inplace = True)

In [ ]:
# 필요 없는 부분 제거
df.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
# X와 y를 나누기 위해 list 생성
x_data = list(df.columns)
x_data.remove('target')

In [ ]:
processing = ['annual_inc',
 'dti',
 'installment',
 'int_rate',
 'loan_amnt',
 'mort_acc',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'num_accts_ever_120_pd',
 'pub_rec',
 'pub_rec_bankruptcies',
 'emp_length_int',
 'issue_d_year',
 'issue_d_month',
 'grade_numeric',
 'fico_range_mean']

# 이거만 바꿀지 다 바꿀지, 의견 물어보고 나의 의견도 제시하면 될듯.

dummy = ['home_ownership_MORTGAGE',
 'home_ownership_NONE',
 'home_ownership_OTHER',
 'home_ownership_OWN',
 'home_ownership_RENT',
 'purpose_credit_card',
 'purpose_debt_consolidation',
 'purpose_educational',
 'purpose_home_improvement',
 'purpose_house',
 'purpose_major_purchase',
 'purpose_medical',
 'purpose_moving',
 'purpose_other',
 'purpose_renewable_energy',
 'purpose_small_business',
 'purpose_vacation',
 'purpose_wedding',
 'term_ 60 months',
 'addr_state_AL',
 'addr_state_AR',
 'addr_state_AZ',
 'addr_state_CA',
 'addr_state_CO',
 'addr_state_CT',
 'addr_state_DC',
 'addr_state_DE',
 'addr_state_FL',
 'addr_state_GA',
 'addr_state_HI',
 'addr_state_IA',
 'addr_state_ID',
 'addr_state_IL',
 'addr_state_IN',
 'addr_state_KS',
 'addr_state_KY',
 'addr_state_LA',
 'addr_state_MA',
 'addr_state_MD',
 'addr_state_ME',
 'addr_state_MI',
 'addr_state_MN',
 'addr_state_MO',
 'addr_state_MS',
 'addr_state_MT',
 'addr_state_NC',
 'addr_state_ND',
 'addr_state_NE',
 'addr_state_NH',
 'addr_state_NJ',
 'addr_state_NM',
 'addr_state_NV',
 'addr_state_NY',
 'addr_state_OH',
 'addr_state_OK',
 'addr_state_OR',
 'addr_state_PA',
 'addr_state_RI',
 'addr_state_SC',
 'addr_state_SD',
 'addr_state_TN',
 'addr_state_TX',
 'addr_state_UT',
 'addr_state_VA',
 'addr_state_VT',
 'addr_state_WA',
 'addr_state_WI',
 'addr_state_WV',
 'addr_state_WY']

### 1. Standard Scaler에 따른 결과
dummy, target을 제외한 부분을 scale 처리

In [ ]:
# standard로 scale
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[processing])

# DF에 스케일된 데이터를 저장 # 아직은 수익률 계산을 하지 않고 parameter만 뽑을 것이므로 기존 데이터를 날려도 큰 상관X
df[processing] = scaled_data

In [ ]:
# 우선 랜덤하게 data_split하고 돌려봄
df_train, df_test = train_test_split(df, test_size = .2, train_size = .8, random_state = 42)

In [ ]:
# train셋을 oversampling
from imblearn.over_sampling import SMOTE
smt = SMOTE()
X_train, y_train = smt.fit_resample(df_train[x_data], df_train['target'])

In [ ]:
X_test = df_test[x_data]
y_test = df_test['target']
# Target

In [ ]:
# XGB
param_xgb = {"max_depth": [10,20,30],
              "min_child_weight" : [3,6],
              "n_estimators": [100,150,200],
             "learning_rate" : [0.1,0.2,0.3],
             'penalty':['ridge','lasso']
              }
# LGB
param_lgb = {"max_depth" : [10,20,30],
             "num_leaves" : [50,100,200],
             "n_estimators": [100,150,200],
             "learning_rate" : [0.1,0.2,0.3],
             'penalty':['ridge','lasso']
              }

# CAT
param_cat = {"depth" : [4,5,6],
          "iterations" : [100, 150,200],
          "learning_rate" : [0.1,0.2,0.3],
          "l2_leaf_reg" : [5,10,15,20], # cat은 기본적으로 L2규제를 사용
          }

In [ ]:
xgb_gs = GridSearchCV(estimator=XGBClassifier(), param_grid=param_xgb, scoring='accuracy', cv=2, refit=True, n_jobs=-1, verbose=2)
xgb_gs.fit(X_train[:200000], y_train[:200000])

# 학습된 모델의 속성 확인
print("Best hyperparameters:", xgb_gs.best_estimator_.get_params())

Fitting 2 folds for each of 108 candidates, totalling 216 fits
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=ridge; total time=   2.8s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=ridge; total time=   3.3s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=lasso; total time=   1.2s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=lasso; total time=   1.2s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=150, penalty=ridge; total time=   1.4s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=150, penalty=ridge; total time=   1.4s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=150, penalty=lasso; total time=   1.5s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=150, penalty=lasso; total time=   1.4s
[CV] END learning_rate=0.

In [ ]:
lgb_gs = GridSearchCV(estimator=LGBMClassifier(), param_grid=param_lgb, scoring='accuracy', cv=3, refit=True, n_jobs=3, verbose=2)
lgb_gs.fit(X_train[:200000], y_train[:200000])

# 학습된 모델의 속성 확인
print("Best hyperparameters:", lgb_gs.best_estimator_.get_params())

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[LightGBM] [Warning] Unknown parameter: penalty
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Unknown parameter: penalty
[LightGBM] [Info] Number of positive: 87709, number of negative: 112291
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.006204 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2093
[LightGBM] [Info] Number of data points in the train set: 200000, number of used features: 83
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.438545 -> initscore=-0.247069
[LightGBM] [Info] Start training from score -0.247069
Best hyperparameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': 10, 'min_child_samples': 20, 'min_child_wei

In [ ]:
print("Best hyperparameters:", lgb_gs.best_estimator_.get_params())

Best hyperparameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': 10, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': None, 'num_leaves': 50, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'penalty': 'ridge'}


In [ ]:
cat_gs2 = GridSearchCV(estimator=CatBoostClassifier(), param_grid=param_cat, scoring='accuracy', cv=3, refit=True, n_jobs=-1, verbose=2)
cat_gs2.fit(X_train[:200000], y_train[:200000])

# 학습된 모델의 속성 확인
print("Best hyperparameters:", cat_gs2.best_estimator_.get_params())

Fitting 3 folds for each of 108 candidates, totalling 324 fits
0:	learn: 0.6499921	total: 66ms	remaining: 13.1s
1:	learn: 0.6161551	total: 77.5ms	remaining: 7.67s
2:	learn: 0.5912619	total: 89.2ms	remaining: 5.86s
3:	learn: 0.5698716	total: 101ms	remaining: 4.92s
4:	learn: 0.5519952	total: 112ms	remaining: 4.37s
5:	learn: 0.5387455	total: 123ms	remaining: 3.97s
6:	learn: 0.5284783	total: 133ms	remaining: 3.67s
7:	learn: 0.5201882	total: 144ms	remaining: 3.46s
8:	learn: 0.5131315	total: 155ms	remaining: 3.29s
9:	learn: 0.5076539	total: 166ms	remaining: 3.15s
10:	learn: 0.5034552	total: 176ms	remaining: 3.02s
11:	learn: 0.4992297	total: 187ms	remaining: 2.92s
12:	learn: 0.4960711	total: 197ms	remaining: 2.83s
13:	learn: 0.4934669	total: 207ms	remaining: 2.75s
14:	learn: 0.4911127	total: 218ms	remaining: 2.69s
15:	learn: 0.4886892	total: 229ms	remaining: 2.63s
16:	learn: 0.4866763	total: 241ms	remaining: 2.59s
17:	learn: 0.4853702	total: 251ms	remaining: 2.54s
18:	learn: 0.4839396	total: 

In [ ]:
print("Best hyperparameters:", cat_gs2.best_estimator_.get_params())

Best hyperparameters: {'iterations': 200, 'learning_rate': 0.1, 'depth': 6, 'l2_leaf_reg': 5}


In [ ]:
cat_gs = GridSearchCV(estimator=CatBoostClassifier(), param_grid=param_xgb, scoring='accuracy', cv=3, refit=True, n_jobs=1, verbose=2)
cat_gs.fit(X_train[:200000], y_train[:200000])

# 학습된 모델의 속성 확인
print("Best hyperparameters:", cat_gs.best_estimator_.get_params())

Fitting 3 folds for each of 108 candidates, totalling 324 fits
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=ridge; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=ridge; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=ridge; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=lasso; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=lasso; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=100, penalty=lasso; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=150, penalty=ridge; total time=   0.1s
[CV] END learning_rate=0.1, max_depth=10, min_child_weight=3, n_estimators=150, penalty=ridge; total time=   0.1s
[CV] END learning_rate=0.

ValueError: 
All the 324 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
162 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 5220, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2385, in _fit
    train_params = self._prepare_train_params(
  File "/usr/local/lib/python3.10/dist-packages/catboost/core.py", line 2311, in _prepare_train_params
    _check_train_

필요 : `'max_depth': 10, 'min_child_weight': 6, 'n_estimators': 100, 'penalty': 'ridge'


필요 : 'max_depth': 20, 'n_estimators': 150, 'n_jobs': None, 'num_leaves': 31, 'penalty': 'ridge'


필요 : 'max_depth': 20, 'n_estimators': 150, 'learning_rate': 0.1